In [1]:
#######################################################
# 예제 8-17 챗봇 엔진 서버 메인 프로그램
# /bot.py 또는 /bot.ipynb
#######################################################
import sys
sys.path.append('C:/chatbot')

import threading
import json

from config.DatabaseConfig import *
from utils.Database import Database
from utils.BotServer import BotServer
from utils.Preprocess import Preprocess
from models.intent.IntentModel import IntentModel
from models.ner.NerModel import NerModel
from utils.FindAnswer import FindAnswer

In [4]:
# 전처리 객체 생성
p = Preprocess(word2index_dic='train_tools/dict/chatbot_dict.bin',
               userdic='utils/user_dic.tsv')

# 의도 파악 모델
intent = IntentModel(model_name='models/intent/intent_model.h5', preprocess=p)

# 개체명 인식 모델
ner = NerModel(model_name='models/ner/ner_model.h5', preprocess=p)

In [5]:
p

In [6]:
intent

In [7]:
ner

In [8]:
def to_client(conn, addr, params):
    db = params['db']

    try:
        db.connect()  # 디비 연결

        # 데이터 수신
        read = conn.recv(2048)  # 수신 데이터가 있을 때 까지 블로킹
        print('===========================')
        print('Connection from: %s' % str(addr))

        if read is None or not read:
            # 클라이언트 연결이 끊어지거나, 오류가 있는 경우
            print('클라이언트 연결 끊어짐')
            exit(0)


        # json 데이터로 변환
        recv_json_data = json.loads(read.decode())
        print("데이터 수신 : ", recv_json_data)
        query = recv_json_data['Query']

        # 의도 파악
        intent_predict = intent.predict_class(query)
        intent_name = intent.labels[intent_predict]

        # 개체명 파악
        ner_predicts = ner.predict(query)
        ner_tags = ner.predict_tags(query)


        # 답변 검색
        try:
            f = FindAnswer(db)
            answer_text, answer_image = f.search(intent_name, ner_tags)
            answer = f.tag_to_word(ner_predicts, answer_text)

        except:
            answer = "죄송해요 무슨 말인지 모르겠어요. 조금 더 공부 할게요."
            answer_image = None

        send_json_data_str = {
            "Query" : query,
            "Answer": answer,
            "AnswerImageUrl" : answer_image,
            "Intent": intent_name,
            "NER": str(ner_predicts)
        }
        message = json.dumps(send_json_data_str)
        conn.send(message.encode())

    except Exception as ex:
        print(ex)

    finally:
        if db is not None: # db 연결 끊기
            db.close()
        conn.close()

In [9]:
# 질문/답변 학습 디비 연결 객체 생성
db = Database(
    host=DB_HOST, user=DB_USER, password=DB_PASSWORD, db_name=DB_NAME
)
print("DB 접속")

DB 접속


In [10]:
port = 5050
listen = 100

# 봇 서버 동작
bot = BotServer(port, listen)
bot.create_sock()
print("bot start")

bot start


In [ ]:
while True:
    conn, addr = bot.ready_for_client()
    params = {
        "db": db
    }

    client = threading.Thread(target=to_client, args=(
        conn,
        addr,
        params
    ))
    client.start()

chatbot 연결 시작
DB 연결 성공!!!
Connection from: ('127.0.0.1', 64733)
데이터 수신 :  {'Query': '안녕하세요?', 'BotType': 'MyService'}
============== Database.py ===================
## Database.py의 select_one함수, sql=
============== FindAnswer.py ===================
** intent_name = 인사
** ner_tags =  []
** sql =  SELECT * FROM chatbot_train_data WHERE intent="인사"  order by rand() limit 1
** answer = {'id': 1, 'intent': '인사', 'ner': None, 'query': '안녕하세요', 'answer': '네 안녕하세요 :D\n반갑습니다. 저는 챗봇입니다.', 'answer_image': 'https://i.imgur.com/UluUFMp.jpg'}
answer= {'id': 1, 'intent': '인사', 'ner': None, 'query': '안녕하세요', 'answer': '네 안녕하세요 :D\n반갑습니다. 저는 챗봇입니다.', 'answer_image': 'https://i.imgur.com/UluUFMp.jpg'}
DB가 닫혔습니다.
chatbot 연결 시작
DB 연결 성공!!!
Connection from: ('127.0.0.1', 64738)
데이터 수신 :  {'Query': '밥 먹으러 갈까요?', 'BotType': 'MyService'}
============== Database.py ===================
## Database.py의 select_one함수, sql=
============== FindAnswer.py ===================
** intent_name = 주문
** ner_tags =  ['B_FOOD

chatbot 연결 시작
DB 연결 성공!!!
Connection from: ('127.0.0.1', 64821)
데이터 수신 :  {'Query': 'exit', 'BotType': 'MyService'}
============== Database.py ===================
## Database.py의 select_one함수, sql=
============== FindAnswer.py ===================
** intent_name = 주문
** ner_tags =  []
** sql =  SELECT * FROM chatbot_train_data WHERE intent="주문"  order by rand() limit 1
** answer = {'id': 3, 'intent': '주문', 'ner': 'B_FOOD', 'query': '{B_FOOD} 주문할게요', 'answer': '{B_FOOD} 주문 처리 완료되었습니다. \n주문해주셔서 감사합니다.', 'answer_image': None}
answer= {'id': 3, 'intent': '주문', 'ner': 'B_FOOD', 'query': '{B_FOOD} 주문할게요', 'answer': '{B_FOOD} 주문 처리 완료되었습니다. \n주문해주셔서 감사합니다.', 'answer_image': None}
DB가 닫혔습니다.
